<a href="https://colab.research.google.com/github/hyunaeee/KU-SW-Academy/blob/main/1016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
TextMining - Information Retrieval
User's information needs 어떻게?
Filterling vs. Ranking
-> Boolean; True or False => 부분집합;Query 잘 작성;모든사람X;연관성(관련성)X
-> Ranking; Relevance높음=Similarity높음; Top-K; Query 고민 X
Similarity     Probabilities(Okapi;BM25)      Inference(NN)X
                    TF-IDF변형
VectorSpace Model
V = {t1,t2,..., tm} ; Controlled Vocabulary
Q = {w1, w2, ..., wn ㅌ V} ; Short Document
C = {d1, d2, ..., dk}
di = {wi1, wi2, ..., win ㅌ V}
Rel(Q, D) = Sim(Q,D) => Sim(Rep(Q),Rep(D))

Inverted Index(g/dbml => FileSystem;B-tree) => Special structure
; A = np.zeros((100000000,100000000))
; A[0,:] => one vector * A => 터짐
X => O(|Q|*D*|D|) => DTM
O => O(|Q|*|L|) => TDM
        V  => Posting(LinkedList-File; Data append; on-disk)
     Vocabulary(Heaps;in-memory) => Feature
Dictionary(K:V-Posting=>File)
IndexTable(색인표) => D1:A    ....     12
                    D2:A ...

                    T1: (D1,D2 ...) => File LinkedList
                    T2: ...
; Single Machine Processing
Map-Reduce => GFS, Hadoop => Spark => BigData
(K,V), (합치는)
=> Local, Global Merge
%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202023-10-16%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%2010.28.27.png

from math import log

tf1 = lambda tf:1 if tf > 0 else 0
tf2 = lambda tf:tf
tf3 = lambda tf, ttf:tf/ttf
tf4 = lambda tf:log(1+tf)
tf5 = lambda tf, mtf, K=0.5:K+(1-K)*(tf/mtf)
import matplotlib.pyplot as plt

TF = [10, 20, 30, 40, 50]
TTF = sum(TF)
MTF = max(TF)
plt.plot(TF, list(map(tf1, TF)), 'r-')
# plt.plot(TF, list(map(tf2, TF)), 'g-')
plt.plot(TF, list(map(tf3, TF, [TTF]*len(TF))), 'b-')
# plt.plot(TF, list(map(tf4, TF)), 'c-')
plt.plot(TF, list(map(tf5, TF, [MTF]*len(TF))), 'm-')
plt.plot(TF, list(map(tf5, TF, [MTF]*len(TF), [0]*len(TF))), 'y-')
[<matplotlib.lines.Line2D at 0x11adc8e50>]

image.png

idf1 = lambda df:1
idf2 = lambda df, N:log(N/df)
idf3 = lambda df, N:log(N/(1+df))+1
idf4 = lambda df, mdf:log(mdf/(1+df))
idf5 = lambda df, N:log((N-df)/df)
DF = [10, 20, 30, 40, 50]
N = max(DF)+1
MDF = max(DF)

plt.plot(DF, list(map(idf1, DF)), 'r-')
plt.plot(DF, list(map(idf2, DF, [N]*len(DF))), 'g-')
plt.plot(DF, list(map(idf3, DF, [N]*len(DF))), 'b-')
plt.plot(DF, list(map(idf4, DF, [MDF]*len(DF))), 'c-')
# plt.plot(DF, list(map(idf5, DF, [N]*len(DF))), 'm-')
[<matplotlib.lines.Line2D at 0x11f9b7dc0>]

       +   *
       |  / (1사분면)
       | /
       |/         Controlled Voca. => Token(1개 이상)
-------0---+----- Concept=Dimension=Orthogonal=Independent=BoW
      /|   weight=importance / each dimension
     / |
    *  |
       |
# [고빈도]
# 1. TF 높은데, DF 높은
# 2.          DF 낮은   ***
# [중빈도]
# 3. TF 평균, DF 평균
# [저빈도]
# 4. TF 낮은데, DF 높은
# 5.         DF 낮은 ***
TF = [50, 50, 25, 10, 10]
DF = [50, 10, 25, 50, 10]

X = list(zip(TF, DF))
# 5(k=0), 5(k=.5)
# 2

plt.plot(range(len(TF)), [tf5(tf, 50, 0)*idf1(df)
                          for tf, df in X], 'r-')
plt.plot(range(len(TF)), [tf5(tf, 50, 0)*idf2(df, 50)
                          for tf, df in X], 'g-')
plt.plot(range(len(TF)), [tf5(tf, 50, 0)*idf3(df, 50)
                          for tf, df in X], 'b-')
plt.plot(range(len(TF)), [tf5(tf, 50, 0)*idf4(df, 50)
                          for tf, df in X], 'k-')
[<matplotlib.lines.Line2D at 0x11f9fe5e0>]

Dictionary + Posting
{term:파일위치} [124][123][123]124123123123
              어느문서, 몇번나왔다=freq, 다음위치
                      가중치=weight, 다음위치

TF5: freq(포스팅 있음), maxfreq(있긴 한데, 다 뒤져야 함), K(우리가 정함)
IDF2: docFreq(-1까지 몇번 돌았는지), N(len(Collection))

t1 = {d1, d2, ..., dn}
      0   10        0 => Sparse to Dense
           w* => TF-IDF
t2 = {         }
d2
|   t1
|  /  t2
| /  /
|/  /
------------ d1
import re
from nltk.tokenize import regexp_tokenize
from os import listdir
from struct import pack, unpack

def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(''.join(s[i:i+n]))
    return rst

def fileids(path, ext='txt'):
    files = list(filter(lambda f:re.search(f'{ext}$', f),
                       listdir(path)))
    return list(map(lambda f:f'{path}/{f}', files))

p1 = re.compile(r'\s+')

D = list() # {file:문서1, maxfreq:뭐고}, 문서, ...
V = list()
TDM = dict() # {term1:[위치, df]}

Posting = open('posting.dat', 'wb')
Posting.close()

for file in fileids('naver'):
    i = len(D)
    D.append({'filename':file, 'maxfreq':0})

    with open(file, 'r', encoding='utf8') as fp:
        corpus = fp.read()

    localTDM = dict()
    localPosting = open('local.dat', 'wb')

    # Tokenizing+Normalizing
    for t in regexp_tokenize(p1.sub(' ', corpus), r'\b\w+\b'):
        for g in ngram(t):
            # 문서 1개 작업하는 중에, 단어가 처음 등장
            if g not in localTDM.keys():
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1, -1))
                localTDM[g] = pos
            else:
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1, localTDM[g]))
                localTDM[g] = pos

    localPosting.close()

    # Update; Local -> Global
    Posting = open('posting.dat', 'ab')
    localPosting = open('local.dat', 'rb')

    # k=단어, v=파일위치
    maxFreq = 0
    for k, v in localTDM.items():
        if k not in V:
            V.append(k)

        j = V.index(k)

        freq = 0
        pos = v
        while pos != -1:
            localPosting.seek(pos)
            f, npos = unpack('ii', localPosting.read(8))
            pos = npos
            freq += 1

        if freq > maxFreq:
            maxFreq = freq

        if j not in TDM.keys():
            pos = Posting.tell()
            Posting.write(pack('iii', i, freq, -1))
            TDM[j] = {'fp':pos, 'df':1}
        else:
            pos = Posting.tell()
            Posting.write(pack('iii', i, freq, TDM[j]['fp']))
            TDM[j]['fp'] = pos
            TDM[j]['df'] += 1

    D[i]['maxfreq'] = maxFreq

    localPosting.close()
    Posting.close()
i2w = lambda i:V[i]
w2i = lambda w:V.index(w)

i2d = lambda i:D[i]
d2i = lambda d:D.index(d)
query = '전손차량 중 확인되지 않는 차량이 4만여대에 달한다는 지적이 나왔다. 전손차량은 자동차가 완전히 파손됐거나 침수 등으로 수리할 수 없는 상태인 자동차와 발생한 손해액이 보험가액 이상인 자동차를 말한다.'

result = list()

Posting = open('posting.dat', 'rb')

for t in regexp_tokenize(query, r'\b\w+\b'):
    for g in ngram(t): # ngram => tokenizing
        if g in V:
            result.append(list())

            j = w2i(g)
            pos = TDM[j]['fp']
            while pos != -1:
                Posting.seek(pos)
                i, freq, npos = unpack('iii', Posting.read(12))
                pos = npos

                result[-1].append(i)
            result[-1] = list(set(result[-1]))

Posting.close()
WDM = dict()
Posting = open('posting.dat', 'rb')
Weighting = open('weighting.dat', 'wb')

for k, v in TDM.items():
    fp = Weighting.tell()
    WDM[k] = {'fp':fp, 'df':v['df']}

    pos = v['fp']
    while pos != -1:
        Posting.seek(pos)
        i, freq, npos = unpack('iii', Posting.read(12))

        tf = tf5(freq, D[i]['maxfreq'], 0)
        idf = idf2(v['df'], len(D))
        w = tf*idf
        Weighting.write(pack('if', i, w))

        pos = npos

Posting.close()
Weighting.close()
Posting
어느문서,몇번,다음위치*

df=3
Weighting
fp:어느문서,가중치,어느문서,가중치,어느문서,가중치 => Sequential
query = '전손차량 중 확인되지 않는 차량이 4만여대에 달한다는 지적이 나왔다. 전손차량은 자동차가 완전히 파손됐거나 침수 등으로 수리할 수 없는 상태인 자동차와 발생한 손해액이 보험가액 이상인 자동차를 말한다.'

Weighting = open('weighting.dat', 'rb')

for t in regexp_tokenize(query, r'\b\w+\b'):
    for g in ngram(t): # ngram => tokenizing
        if g in V:
            j = w2i(g)

            n = 0
            while n < WDM[j]['df']:
                Weighting.seek(WDM[j]['fp']+n*8)
                i, w = unpack('if', Weighting.read(8))
                pos = npos
                n += 1

                print(f'{i}번째 문서, {j}번째 단어 {g}의 가중치 {w}')

Weighting.close()